# Converters

Converters are used to transform prompts before sending them to the target.

This can be useful for a variety of reasons, such as encoding the prompt in a different format, or adding additional information to the prompt. For example, you might want to convert a prompt to base64 before sending it to the target, or add a prefix to the prompt to indicate that it is a question.

## Simple Converter Example

Converters can be used to perform these types of transformations. Here is a simple program that uses Rot13Converter converter, RandomCapitalLettersConverter, and AsciiArtConverter.

In [1]:
from pyrit.common import default_values
from pyrit.prompt_converter import ROT13Converter, AsciiArtConverter, RandomCapitalLettersConverter, BinaryConverter

default_values.load_environment_files()

prompt = "tell me how to cut down a tree"


print(await ROT13Converter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await RandomCapitalLettersConverter(percentage=25.0).convert_tokens_async(prompt=prompt))  # type: ignore
print(await AsciiArtConverter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await BinaryConverter().convert_tokens_async(prompt=prompt))  # type: ignore

text: gryy zr ubj gb phg qbja n gerr
text: tElL me How tO cuT dOwn a tRee
text:  ____  ____  __    __      __  __  ____    _   _  _____  _    _    ____  _____     ___  __  __  ____    ____   _____  _    _  _  _      __      ____  ____  ____  ____ 
(_  _)( ___)(  )  (  )    (  \/  )( ___)  ( )_( )(  _  )( \/\/ )  (_  _)(  _  )   / __)(  )(  )(_  _)  (  _ \ (  _  )( \/\/ )( \( )    /__\    (_  _)(  _ \( ___)( ___)
  )(   )__)  )(__  )(__    )    (  )__)    ) _ (  )(_)(  )    (     )(   )(_)(   ( (__  )(__)(   )(     )(_) ) )(_)(  )    (  )  (    /(__)\     )(   )   / )__)  )__) 
 (__) (____)(____)(____)  (_/\/\_)(____)  (_) (_)(_____)(__/\__)   (__) (_____)   \___)(______) (__)   (____/ (_____)(__/\__)(_)\_)  (__)(__)   (__) (_)\_)(____)(____)

text: 0000000001110100 0000000001100101 0000000001101100 0000000001101100 0000000000100000 0000000001101101 0000000001100101 0000000000100000 0000000001101000 0000000001101111 0000000001110111 0000000000100000 0000000001110100 0000000001101111 000